In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [58]:
df = pd.read_csv('Data/df_clean.csv', sep=';', index_col=0)

In [59]:
def to_lower():
    for col in ['NOM de VOIE', 'HISTORIQUE', 'Libellé VOIE']:
        df[col] = df[col].str.lower()
to_lower()

# 1. Charger données Wikipedia

In [60]:
website_url = requests.get('https://fr.wikipedia.org/wiki/Liste_des_voies_de_Paris_se_r%C3%A9f%C3%A9rant_%C3%A0_un_nom_de_femme').text
soup = BeautifulSoup(website_url,'lxml')
tables = soup.find_all('table',class_="wikitable")

In [61]:
def export_wiki_table(table, table_name) :
    voies = []
    eponymes = []
    arrondissements = []
    dates=[]
    for tr in table.find_all('tr'):
            tds = tr.find_all('td')
            if not tds:
                continue
            if len(tds)==3:
                voie, eponyme, arrondissement = [td.text.strip() for td in tds]
            else : 
                voie, eponyme, arrondissement, date = [td.text.strip() for td in tds]
            voies.append(voie)
            eponymes.append(eponyme)
            arrondissements.append(arrondissement)
            if len(tds)==4:
                dates.append(date)
    if len(tds)==3:
        export_df = pd.DataFrame(list(zip(voies, eponymes,arrondissements)), 
                                columns =['Voies', 'Eponymes', 'Arrondissements'])
    else: 
        export_df = pd.DataFrame(list(zip(voies, eponymes,arrondissements, dates)), 
                                columns =['Voies', 'Eponymes', 'Arrondissements', 'Date'])
    export_df['Origine rue'] = table_name
    return(export_df)

In [62]:
table_pers=export_wiki_table(tables[0], "Personnalités")
table_proprio=export_wiki_table(tables[1], "Famille de propriétaires")
table_relig=export_wiki_table(tables[2], "Religion")
table_div=export_wiki_table(tables[3], "Divers")
df_women = pd.concat([table_pers, table_proprio, table_relig, table_div], axis=0, sort=False).reset_index(drop=True)

In [63]:
# Cleaning 
for col in ['Voies', 'Eponymes']:
    df_women[col] = df_women[col].str.lower()

df_women['Voies'] = df_women['Voies'].astype(str).apply(lambda x: x.replace('-', ' '))
df_women['Voies'] = df_women['Voies'].astype(str).apply(lambda x: x.replace('œ', 'oe'))
df_women['Voies'] = df_women['Voies'].astype(str).apply(lambda x: x.replace('et ', ''))
df_women['Voies'] = df_women['Voies'].astype(str).apply(lambda x: x.replace('á', 'a'))

    
# Ajouter infos supplémentaires quand c'est possible
df_women.loc[df_women.index==2, 'Origine_rue'] = 'actrice'
df_women.loc[df_women.index==23, 'Date_personnalité'] = '1648-1723'

# A nettoyer: fautes d'orthographe
df_women.loc[df_women['Voies'].str.contains('veil'), 'Voies'] = "place de l'europe simone veil"
df_women.loc[df_women['Voies'].str.contains('viera'), 'Voies'] = "rue maria helena vieira da silva"


In [64]:
# Clean quelques noms
df.loc[df['Libellé VOIE'].str.contains('desbordes'), 'Libellé VOIE'] = 'rue marceline desbordes valmore'
df.loc[df['Libellé VOIE'].str.contains('dury'), 'Libellé VOIE'] = 'villa hortense dury vasselon'
df.loc[df['Libellé VOIE'].str.contains('heyman'), 'Libellé VOIE'] = 'parvis claire heyman maria errazuriz'
df.loc[df['Libellé VOIE'].str.contains('auvergne'), 'Libellé VOIE'] = df.loc[df['Libellé VOIE'].str.contains('auvergne'), 'Libellé VOIE'].astype(str).apply(lambda x: x.replace("de la tour d'auvergne", "louise émilie de la tour d'auvergne"))
df.loc[df['Libellé VOIE'].str.contains('skobtsov'), 'Libellé VOIE'] = 'rue mère marie skobtsov'
df.loc[df['Libellé VOIE'].str.contains('mercouri'), 'Libellé VOIE'] = 'place melina mercouri'
df.loc[df['Libellé VOIE'].str.contains('millerde'), 'Libellé VOIE'] = 'avenue anne eugénie millerde brou'
df.loc[df['Libellé VOIE'].str.contains('moreau evrard'), 'Libellé VOIE'] = 'terrasse émilienne moreau évrard'
df.loc[df['Libellé VOIE'].str.contains('rue récamier'), 'Libellé VOIE'] = 'rue juliette récamier'
df.loc[df['Libellé VOIE'].str.contains('rochechouart'), 'Libellé VOIE'] = df.loc[df['Libellé VOIE'].str.contains('rochechouart'), 'Libellé VOIE'].astype(str).apply(lambda x: x.replace('de rochechouart', "marguerite de rochechouart"))
df.loc[df['Libellé VOIE'].str.contains('staël'), 'Libellé VOIE'] = 'rue germaine de staël'
df.loc[df['Libellé VOIE'].str.contains('marie claude vaillant'), 'Libellé VOIE'] = 'place marie claude vaillant couturier pierre villon'
df.loc[df['Libellé VOIE'].str.contains('tréfouel'), 'Libellé VOIE'] = 'place jacques thérèse tréfouël'
df.loc[df['Libellé VOIE'].str.contains('vigée'), 'Libellé VOIE'] = 'rue élisabeth vigée le brun'


# 2. Merger

In [65]:
df = pd.merge(df, df_women, how='left', left_on='Libellé VOIE', right_on='Voies', copy=False)
df = df.drop_duplicates(subset='Libellé VOIE')
#len(df.loc[test_merge['Libellé VOIE'].isna()])

In [66]:
df.loc[~df["Voies"].isna(), "Femme"] = 1

In [67]:
# Ajout à la main : LABELLISER TOUS LES SAINTS
df.loc[(df['Libellé VOIE'].str.contains('sainte foy')), 'Femme'] = 1
df.loc[(df['Libellé VOIE'].str.contains('sainte foy')), 'Origine rue'] = "Religion"

In [68]:
len(df.loc[df['Femme']==1])

331

In [69]:
# Correction : les femmes qui n'avaient pas été labellisées comme des personnes
df.loc[df['Femme']==1, 'bool_personne']=1

# 3. Petits clean

In [70]:
df.loc[(df['bool_personne']==1) & df['Femme'].isna(), 'Homme']=1

In [71]:
df.loc[df['Homme']==1,'Sexe'] = 'M'
df.loc[df['Femme']==1, 'Sexe'] = 'F'

In [72]:
df['Sexe'].value_counts()

M    3514
F     331
Name: Sexe, dtype: int64

In [74]:
# Ajouter profession des femmes si non ajouté
df['Origine_rue'].value_counts()

actrice    1
Name: Origine_rue, dtype: int64

In [77]:
df.loc[df['JOB1'].isna(), 'JOB1'] = df['Origine_rue']
df.loc[df['JOB1'].isna(), 'JOB1'] = df['Origine rue']

In [82]:
df.to_csv('Data/df_final.csv', sep=';')